<a href="https://colab.research.google.com/github/Phignis/predict-women-soccer-results/blob/main/src/predict_soccer_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>